<a href="https://colab.research.google.com/github/wooohun/BERT-Summarizer/blob/main/BART_Abstractive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
# install transformers from huggingface
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# install kaggle
!pip install -q kaggle
!mkdir ~/.kaggle

# get kaggle.json from kaggle website -> create new api token
# add kaggle.json to colab folder
# move kaggle api token to kaggle folder
!cp -v kaggle.json ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists
'kaggle.json' -> '/root/.kaggle/kaggle.json'


In [4]:
# download dataset
# !chmod 600 /root/.kaggle/kaggl
!kaggle datasets download -d gowrishankarp/newspaper-text-summarization-cnn-dailymail
!unzip newspaper-text-summarization-cnn-dailymail

newspaper-text-summarization-cnn-dailymail.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  newspaper-text-summarization-cnn-dailymail.zip
replace cnn_dailymail/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace cnn_dailymail/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace cnn_dailymail/validation.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [5]:
# grab datasets
test = pd.read_csv("cnn_dailymail/test.csv")
train = pd.read_csv("cnn_dailymail/train.csv")
validate = pd.read_csv("cnn_dailymail/validation.csv")

In [6]:
from transformers import AutoTokenizer, BartForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-cnn')
model  = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

In [ ]:
# tokenize articles, highlights for test, train datasets

test['article_tokens'] = test['article'].apply(tokenizer.tokenize, truncation=True, max_length = 1024)
# train['article_tokens'] = train['article'].apply(tokenizer.tokenize)

test['summary_tokens'] = test['highlights'].apply(tokenizer.tokenize)
# train['summary_tokens'] = train['highlights'].apply(tokenizer.tokenize)

In [10]:
test

,id,article,highlights,article_tokens
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...,"[Ever, Ġnoticed, Ġhow, Ġplane, Ġseats, Ġappear..."
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...,"[A, Ġdrunk, Ġteenage, Ġboy, Ġhad, Ġto, Ġbe, Ġr..."
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...,"[Doug, ie, ĠFreed, man, Ġis, Ġon, Ġthe, Ġverge..."
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...,"[Liverpool, Ġtarget, ĠNet, o, Ġis, Ġalso, Ġwan..."
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6...","[Bruce, ĠJenner, Ġwill, Ġbreak, Ġhis, Ġsilence..."
...,...,...,...,...
11485,ed8674cc15b29a87d8df8de1efee353d71122272,Our young Earth may have collided with a body ...,Oxford scientists say a Mercury-like body stru...,"[Our, Ġyoung, ĠEarth, Ġmay, Ġhave, Ġcollided, ..."
11486,2f58d1a99e9c47914e4b1c31613e3a041cd9011e,A man facing trial for helping his former love...,Man accused of helping former lover kill woman...,"[A, Ġman, Ġfacing, Ġtrial, Ġfor, Ġhelping, Ġhi..."
11487,411f6d57825161c3a037b4742baccd6cd227c0c3,A dozen or more metal implements are arranged ...,Marianne Power tried the tuning fork facial at...,"[A, Ġdozen, Ġor, Ġmore, Ġmetal, Ġimplements, Ġ..."
11488,b5683ef8342056b17b068e0d59bdbe87e3fe44ea,Brook Lopez dominated twin brother Robin with ...,Brooklyn Nets beat the Portland Trail Blazers ...,"[Brook, ĠLopez, Ġdominated, Ġtwin, Ġbrother, Ġ..."
